In [2]:
import bempp.api 
import numpy as np
import mtf

from mtf.utils import bhmie
from mtf.config import config
from mtf.functions import define_bempp_functions
from mtf.utils.iterative import gmres
from mtf.preconditioning.osrc import osrc_MtE

from matplotlib import pyplot as plt
from decimal import Decimal

bempp.api.enable_console_logging()
M = 2

mtf.config.set_case("B")

tangential_trace, neumann_trace = define_bempp_functions(config)

k0, k1 = config["k_ext"], config["k_int"]
lambda_par, freq = config["lambda"], config["frequency"]

polarization = config["polarization"]
direction = config["direction"]

eps_rel = config["eps_rel"]
mu_rel = config["mu_rel"]
mu0 = config["mu_ext"]
mu1 = mu_rel * mu0

eta_rel = np.sqrt(mu_rel / eps_rel)

print("The exterior wavenumber is: {0}".format(k0))
print("The interior wavenumber is: {0}".format(k1))

print("----")
print("The exterior wavelenght is: {0}".format(lambda_par))
print("The exterior frequency is: {:.2E}".format(Decimal(freq)))

The exterior wavenumber is: 5.0
The interior wavenumber is: 6.892024376045111
----
The exterior wavelenght is: 1.2566370614359172
The exterior frequency is: 2.39E+8


In [3]:
segments = [[10], [10]]
swapped_normals = [[10], []]

k_int, k_ext = config["k_int"], config["k_ext"]

n = k_int / k_ext
refIndex = n
numAngles = 901
s1, s2, qext, qsca, qback, gsca = bhmie(k_ext, k_int / k_ext, numAngles)
angles = config['angles']

#transmission_operators = assemble_operators(grid, config)
#far_field, solution = evaluate_far_field(transmission_operators, config)

k_list = [k0]
eta_rel_list = [1]
mu_list = [mu0]

for index in range(M-1):
  k_list.append(k1)
  mu_list.append(mu1)
  eta_rel_list.append(eta_rel)

In [4]:
precision = 1

h = 2 * np.pi/(precision*k0)
grid = bempp.api.shapes.sphere(h=h)


print(h, ': h')
print(precision, ': precision')
print(grid.number_of_edges * 2, ': N')

    
dA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

p1dA = [bempp.api.function_space(grid, "DP", 1, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

rA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
tA = [bempp.api.function_space(grid, "SNC", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

multitrace_ops = []
osrc_ops = []

# > Assemble all diagonal operators
for index in range(M):
  k = k_list[index]
  mu = mu_list[index]
  eta = eta_rel_list[index]
  efie = bempp.api.operators.boundary.maxwell.electric_field(dA[1], rA[1], tA[1], k, assembler='fmm')
  osrc = osrc_MtE(dA[1], rA[1], tA[1], p1dA[1], k)
  mfie = bempp.api.operators.boundary.maxwell.magnetic_field(dA[1], rA[1], tA[1], k, assembler='fmm')
  block_osrc = bempp.api.BlockedOperator(2,2)
  block_osrc[0,1] = eta * osrc
  block_osrc[1,0] = -1/eta * osrc
  osrc_ops.append(block_osrc)
  multitrace_ops.append(bempp.api.GeneralizedBlockedOperator([[mfie, eta * efie],[- 1/eta * efie, mfie]]))
  #osrc_ops.append(bempp.api.GeneralizedBlockedOperator([[zero, eta * osrc],[- 1/eta * osrc, zero]]))

lhs_op = multitrace_ops[0] + multitrace_ops[1]    

prec_op = osrc_ops[0] + osrc_ops[1]    

rhs = [bempp.api.GridFunction(rA[1], dual_space = tA[1], fun=tangential_trace),
      bempp.api.GridFunction(rA[1], dual_space = tA[1], fun=neumann_trace)]

b = bempp.api.assembly.blocked_operator.projections_from_grid_functions_list(rhs, lhs_op.dual_to_range_spaces)

P = prec_op.weak_form()
op_wf = lhs_op.weak_form()
x_gmres, conv_gmres, res_gmres = gmres(P * op_wf, P * b, return_residuals=True, restart = 1000)
sol = bempp.api.assembly.blocked_operator.grid_function_list_from_coefficients(x_gmres.ravel(), lhs_op.domain_spaces)
far_field_points = config['far_field_points']
electric_far = bempp.api.operators.far_field.maxwell.electric_field(sol[1].space, far_field_points, k0)
magnetic_far = bempp.api.operators.far_field.maxwell.magnetic_field(sol[0].space, far_field_points, k0)    
far_field = - electric_far * sol[1] - magnetic_far * sol[0]

A22 = far_field[2,:]
uh = 10 * np.log10(4 * np.pi * np.abs(A22[:1801]))
u =  10 * np.log10(4 * np.pi * np.abs(s1 / (-1j * k_ext) ))
rel_error = np.linalg.norm(uh - u) / np.linalg.norm(u)

print(rel_error)

bempp:HOST:INFO: Created grid with id b2f3e601-07cc-4d0f-8ae5-3b98be81019d. Elements: 42. Edges: 63. Vertices: 23
bempp:HOST:WARNING: Setting include_boundary_dofs has no effect on this space type.
bempp:HOST:WARNING: Setting include_boundary_dofs has no effect on this space type.


1.2566370614359172 : h
1 : precision
126 : N


/usr/local/lib/python3.10/dist-packages/bempp/api/assembly/discrete_boundary_operator.py:639: SparseEfficiencyWarning: splu converted its input to CSC format
  solver = solver_interface(actual_mat)
/usr/local/lib/python3.10/dist-packages/bempp/api/assembly/discrete_boundary_operator.py:639: SparseEfficiencyWarning: splu converted its input to CSC format
  solver = solver_interface(actual_mat)
bempp:HOST:INFO: OpenCL CPU Device set to: pthread-AMD EPYC 7302 16-Core Processor
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:499: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  prg.build(options_bytes, devices)
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:495: Co

0.3464872598139258
